## Model Training
 #### 1.1 Import Data and Required Packages
##### Importing Pandas, Numpy, Matplotlib, Seaborn and Warings Library.

In [66]:
# Basic Import\n",
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling\n",
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [67]:
df = pd.read_csv('modified_student_details.csv')
# Top 5 record from the dataset
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score,total_score,avg_score
0,female,group B,bachelor's degree,standard,none,72,72,74,218,72.666667
1,female,group C,some college,standard,completed,69,90,88,247,82.333333
2,female,group B,master's degree,standard,none,90,95,93,278,92.666667
3,male,group A,associate's degree,free/reduced,none,47,57,44,148,49.333333
4,male,group C,some college,standard,none,76,78,75,229,76.333333


### Prepare X and Y variable for training and testing.
- I will try to predict writing_score. so i will keep it inside y variable

In [68]:
x = df.drop('writing_score', axis=1)
y = df['writing_score']

In [69]:
x.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,total_score,avg_score
0,female,group B,bachelor's degree,standard,none,72,72,218,72.666667
1,female,group C,some college,standard,completed,69,90,247,82.333333
2,female,group B,master's degree,standard,none,90,95,278,92.666667
3,male,group A,associate's degree,free/reduced,none,47,57,148,49.333333
4,male,group C,some college,standard,none,76,78,229,76.333333


In [70]:
# Categorical features and uniqueness
numerical_features = ['math_score', 'reading_score', 'writing_score', 'total_score', 'avg_score']

for i in df.columns:
        if i not in numerical_features:
            print(f'Categories in {i} are {df[i].unique()}')

Categories in gender are ['female' 'male']
Categories in race_ethnicity are ['group B' 'group C' 'group A' 'group D' 'group E']
Categories in parental_level_of_education are ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
Categories in lunch are ['standard' 'free/reduced']
Categories in test_preparation_course are ['none' 'completed']


### Break down categorical and numerical features

In [71]:
# Create Column Transformer with 3 types of transformers\n"
num_features = x.select_dtypes(exclude="object").columns
cat_features = x.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

num_transformer = StandardScaler() # for numerical features
cat_transformer = OneHotEncoder() # for categorical features



preprocessor = ColumnTransformer( # ColumnTransformer responsible for transforming matrix
    [
        ("OneHotEncoder", cat_transformer, cat_features),
         ("StandardScaler", num_transformer, num_features),        
    ]
)

In [72]:
x = preprocessor.fit_transform(x)

In [73]:
x

array([[ 1.        ,  0.        ,  0.        , ...,  0.19399858,
         0.34357423,  0.34357423],
       [ 1.        ,  0.        ,  0.        , ...,  1.42747598,
         1.0219275 ,  1.0219275 ],
       [ 1.        ,  0.        ,  0.        , ...,  1.77010859,
         1.74706375,  1.74706375],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.12547206,
        -0.19443008, -0.19443008],
       [ 1.        ,  0.        ,  0.        , ...,  0.60515772,
         0.46053169,  0.46053169],
       [ 1.        ,  0.        ,  0.        , ...,  1.15336989,
         1.06871048,  1.06871048]])

In [79]:
x.shape

(1000, 21)

In [82]:
# Train test split

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.2, random_state=42)

x_train.shape

(200, 21)

### Model Evaluation



In [83]:
def evaluation(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mse(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_score

In [84]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]
